In [174]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import csv
import re

import numpy as np
import math

from sklearn import preprocessing

## Extrating the Train Data

In [176]:
df = pd.DataFrame(pd.read_json('distiller_20200330.json'))
df.insert(0,'id', range(len(df)))
df.reset_index(drop=True)
df.sample(5)

,id,abv,age,average_user_rating,brand_name,cost_level,description,expert_rating,flavor_profile,name,origin,reviewer,tasting_notes,type,user_comments
1954,1954,40.0,NaN,NaN,Puerto Angel,2,Produced in a distillery located in the Oaxaca...,86.0,"{'earthy': 0, 'floral': 20, 'fruity': 40, 'her...",Puerto Angel Blanco Rum,Mexico,Paul Senft,The aroma delivers an interesting balance of m...,Silver Rum,0
7215,7215,48.0,NaN,77.0,Kilchoman,4,This vintage bottling from Kilchoman is a vatt...,91.0,"{'briny': 25, 'floral': 20, 'fruity': 30, 'ful...",Kilchoman Vintage 2010,"Islay, Scotland",Jacob Grier,This is assertively peated whisky with some oo...,Peated Single Malt,5
5376,5376,40.0,NaN,66.8,E & J,1,E&J distillers is owned by the Ernest and Juli...,NaN,None,E&J XO Brandy,"California, USA",None,None,American Brandy,38
2643,2643,46.0,NaN,67.6,Finlaggan,2,"Family-run independent bottler, The Vintage Ma...",NaN,None,Finlaggan Eilean Mòr,"Islay, Scotland",None,None,Peated Single Malt,31
4591,4591,50.0,NaN,68.0,Heaven Hill,1,"Old Fitzgerald, a product of Heaven Hills Dist...",84.0,"{'briny': 0, 'floral': 40, 'fruity': 64, 'full...",Old Fitzgerald Bottled In Bond Bourbon,"Kentucky, USA",Ryan Conklin,The nose on this whiskey is round with a good ...,Bourbon,75


In [177]:
df.to_csv('distiller_20200330.csv', index=False)

In [178]:
df = df[(df['tasting_notes'].isnull() == False) & (df['flavor_profile'].isnull() == False)]
print(len(df))

df = df[['id','type','expert_rating','tasting_notes','flavor_profile']]
df.head(5)

3513


,id,type,expert_rating,tasting_notes,flavor_profile
5,5,Canadian,95.0,This rye beast is crammed with so many spices ...,"{'briny': 0, 'floral': 45, 'fruity': 45, 'full..."
6,6,American Brandy,75.0,The aromatics of this brandy have a strong app...,"{'earthy': 5, 'floral': 10, 'fruity': 75, 'her..."
7,7,Old Tom Gin,90.0,Sweet spices and floral aromas are found as yo...,"{'bitter': 10, 'chemical': 0, 'earthy': 45, 'f..."
11,11,Other Whiskey,84.0,"The nose is floral, with honey, hay, citrus ze...","{'briny': 0, 'floral': 50, 'fruity': 20, 'full..."
12,12,Flavored Whiskey,84.0,The nose is at first difficult to find behind ...,"{'briny': 0, 'floral': 10, 'fruity': 30, 'full..."


In [179]:
df.sample(5)

,id,type,expert_rating,tasting_notes,flavor_profile
2969,2969,Navy Rum,80.0,This product's strong aroma is flush with note...,"{'earthy': 75, 'floral': 40, 'fruity': 20, 'he..."
591,591,Amaro,90.0,Cynar's deep amber color accurately hints at i...,"{'bitter': 95, 'earthy': 50, 'floral': 0, 'fru..."
1159,1159,London Dry Gin,87.0,The aroma is much more aromatic with the sweet...,"{'bitter': 10, 'chemical': 0, 'earthy': 25, 'f..."
4944,4944,Canadian,87.0,This whisky hits hard with a sweet nose brimmi...,"{'briny': 0, 'floral': 30, 'fruity': 45, 'full..."
1113,1113,Single Malt,86.0,Israel isn't the first country one thinks of f...,"{'briny': 30, 'floral': 25, 'fruity': 30, 'ful..."


### Exporting as new CSV

In [180]:
df.to_csv('train_raw.csv')

## Re-organize the Data

In [181]:
id_ = []
name = []
flavor_profile = []
tasting_notes = []
expert_rating = []
type_ = []

flavor_data = []

with open('train_raw.csv', 'r', newline='', encoding="utf-8") as file:
    reader = csv.reader(file)
    
    for row in reader: 
        
        i = row[0]
        n = row[1]
        c = row[2]
        flavor = row[5].strip('{').strip('}\r').split(',')
        note = row[4].strip()
        rating = row[3]
        
        flavors = []
        
        for item in flavor:
            items = item.strip().split(':')

            flavors.append(items)              
        
        id_.append(i)
        name.append(n)
        flavor_data.append(flavors)
        tasting_notes.append(note)
        expert_rating.append(rating)
        type_.append(c)
        
        
del id_[0]
del name[0]
del type_[0]
del flavor_data[0]  
del tasting_notes[0]
del expert_rating[0]

for line in flavor_data:
    flavor_profile.append(dict(line))

## Allocating Type to Category

In [182]:
type_list = []

for item in type_:
    type_list.append(item)

type_list = list(set(type_list))

print(type_list)

['Flavored Whiskey', 'Other Brandy', 'Sotol Blanco', 'Tequila Mixto Blanco', 'Rhum Agricole Éléve Sous Bois', 'Flavored Rum', 'Rye', 'Mezcal Joven', 'Aged Rum', 'Cognac', 'Barrel-Aged Gin', 'Eau-de-Vie', 'Flavored Gin', 'Spiced Rum', 'Tequila Añejo', 'Unflavored Vodka', 'Blended Grain', 'Calvados', 'Wheat Whiskey', 'Other Agave', 'Chocolate Liqueurs', 'Old Tom Gin', 'Blended', 'Sloe Gin & Variants', 'Pisco', 'Silver Rum', 'Genever-Style', 'Navy-Strength Gin', 'Bourbon', 'Armagnac', 'London Dry Gin', 'Single Pot Still', 'Tennessee', 'Coffee Liqueurs', 'Distilled Gin', 'Modern Gin', 'Single Grain', 'Other Liqueurs', 'White', 'Rhum Agricole Blanc', 'Nut Liqueurs', 'Spirit', 'Gold Rum', 'Tequila Mixto Gold', 'Floral Liqueurs', 'Other Sugarcane', 'Canadian', 'American Brandy', 'Tequila Reposado', 'Blended American Whiskey', 'Corn', 'Single Malt', 'Blended Malt', 'Bitter Liqueurs', 'Genever', 'Triple Sec/Curaçao', 'Grappa/Marc', 'Fruit Liqueurs', 'Peated Single Malt', 'Herbal/Spice Liqueurs'

In [183]:
category= []

category_list = ['Whiskey', 'Brandy', 'Rum', 'Gin', 'Vodka', 'Tequila', 'Liqueurs/Bitters']

whiskey_list = ['Blended Malt', 'Tennessee', 'Peated Single Malt', 'Single Grain', 'Blended', 'Flavored Whiskey', 'Corn', 
                'American Single Malt', 'Blended Grain', 'Blended American Whiskey', 'Rye', 'Bourbon', 'Other Whiskey',
                'Single Malt', 'Wheat Whiskey', 'White', 'Peated Blend', 'Peated Blended Malt', 'Spirit', 'Single Pot Still',
                'Canadian', ]

brandy_list = ['Eau-de-Vie', 'Other Brandy', 'Grappa/Marc',  'Cognac', 'Armagnac', 'Calvados', 'Blanche Armagnac', 'American Brandy',
              'Pisco', ]

rum_list = ['Rhum Agricole Éléve Sous Bois', 'Spiced Rum', 'Gold Rum', 'Aged Rum', 'Aguardiente', 'Flavored Rum', 'Navy Rum',
            'Rhum Agricole Blanc', 'Cachaça', 'Dark Rum', 'Silver Rum', 'Rhum Agricole Vieux', 'Other Sugarcane']

gin_list = ['Distilled Gin', 'Flavored Gin', 'Old Tom Gin', 'Plymouth Gin', 'London Dry Gin', 'Navy-Strength Gin', 
            'Gin de Mahón', 'Compound Gin', 'Genever', 'Modern Gin', 'Genever-Style', 'Barrel-Aged Gin']

vodka_list = ['Unflavored Vodka', 'Flavored Vodka', ]

tequila_list = ['Tequila Añejo', 'Tequila Blanco', 'Tequila Mixto Blanco', 'Tequila Joven', 'Tequila Extra Añejo', 'Mezcal Joven',
               'Raicilla Blanco', 'Tequila Mixto Gold', 'Mezcal Reposado', 'Other Agave', 'Sotol Blanco', 'Tequila Reposado',
               'Sotol Añejo', 'Mezcal Añejo', ]

liqueur_list = ['Bitter Liqueurs', 'Sloe Gin & Variants', 'Chocolate Liqueurs', 'Other Liqueurs', 'Herbal/Spice Liqueurs',
                'Coffee Liqueurs', 'Amaro', 'Triple Sec/Curaçao', 'Fruit Liqueurs', 'Nut Liqueurs', 'Floral Liqueurs', 
                'Dairy/Egg Liqueurs']

for item in type_:

    if item in whiskey_list:
        category.append(category_list[0])
    
    elif item in brandy_list:
        category.append(category_list[1])
        
    elif item in rum_list:
        category.append(category_list[2])
        
    elif item in gin_list:
        category.append(category_list[3])
        
    elif item in vodka_list:
        category.append(category_list[4])
        
    elif item in tequila_list:
        category.append(category_list[5])
        
    else:
        category.append(category_list[6])     

print(type_[-10:-1])            
print(category[-10:-1])

['London Dry Gin', 'Single Malt', 'Gold Rum', 'Cachaça', 'Canadian', 'Spiced Rum', 'American Brandy', 'Amaro', 'Flavored Vodka']
['Gin', 'Whiskey', 'Rum', 'Rum', 'Whiskey', 'Rum', 'Brandy', 'Liqueurs/Bitters', 'Vodka']


## Check the Data

In [184]:
print(len(flavor_profile))
print(type(flavor_profile[-1]))
# print(flavor_profile[0].keys())
# print(flavor_profile[0].values())
print(flavor_profile[-1])

3513
<class 'dict'>
{"'bitter'": ' 0', "'earthy'": ' 10', "'floral'": ' 10', "'fruity'": ' 40', "'herbal'": ' 50', "'mineral'": ' 10', "'oily'": ' 15', "'rich'": ' 30', "'salty'": ' 5', "'smoky'": ' 25', "'spicy'": ' 20', "'sweet'": ' 60', "'tart'": ' 0', "'umami'": ' 5'}


In [185]:
print(len(tasting_notes))
print(type(tasting_notes[-1]))
print(tasting_notes[-1])

3513
<class 'str'>
As unharmonious as those donkeys might be, the flavors of this mezcal are quite the opposite - everything plays together with each pleasurable sip. It has a subtle, sweet and light smoke, with a minty and peppery finish. Roasted agave fruit flavors really shine through with a light, dewy mouthfeel.


In [186]:
print(len(expert_rating))
print(type(expert_rating[0]))
print(expert_rating[0])

3513
<class 'str'>
95.0


## Extracting Flavor Profile Labels

In [187]:
flavor_labels = set()

for keys in flavor_profile:
    for key in keys.keys():
        flavor_labels.add(key)
    
label_list = list(flavor_labels)
print(label_list)
print(len(label_list))

["'rich'", "'chemical'", "'tart'", "'peaty'", "'spicy'", "'floral'", "'bitter'", "'juniper'", "'salty'", "'hogo'", "'nutty'", "'fruity'", "'umami'", "'neutral'", "'rancio'", "'roast'", "'mineral'", "'grain'", "'sweet'", "'briny'", "'vanilla'", "'oily'", "'harsh'", "'full_bodied'", "'earthy'", "'herbal'", "'woody'", "'smoky'"]
28


## Emerging to the New Data

In [188]:
import math

new_data = []

for item in flavor_profile:

    new_keys = []
    new_values = []
    
    for key in item:
    
        for label in label_list:
            if label in item.keys():

                new_keys.append(key.strip("'"))
                new_values.append(math.ceil(int(item[key])/20))
            
            elif label not in item.keys():
                
                new_keys.append(label.strip("'"))
                new_values.append(0)
                
    flavors = dict(zip(new_keys, new_values))
    
    new_data.append(flavors)

In [189]:
print(len(new_data), type(new_data))
print(new_data[-1], type(new_data[-1]))

3513 <class 'list'>
{'bitter': 0, 'chemical': 0, 'peaty': 0, 'juniper': 0, 'hogo': 0, 'nutty': 0, 'neutral': 0, 'rancio': 0, 'roast': 0, 'grain': 0, 'briny': 0, 'vanilla': 0, 'harsh': 0, 'full_bodied': 0, 'woody': 0, 'earthy': 1, 'floral': 1, 'fruity': 2, 'herbal': 3, 'mineral': 1, 'oily': 1, 'rich': 2, 'salty': 1, 'smoky': 2, 'spicy': 1, 'sweet': 3, 'tart': 0, 'umami': 1} <class 'dict'>


In [190]:
for index, item in enumerate(new_data):

    for key in item:
        item[key] = int(item[key])
        #item[key] = flavor_scale[item[key]]
        
    item['id'] = id_[index]
    item['rating'] = round(float(expert_rating[index]))
    item['category'] = category[index]
    item['tasting_notes'] = tasting_notes[index]

In [191]:
new_data[0]

{'briny': 0,
 'chemical': 0,
 'bitter': 0,
 'juniper': 0,
 'hogo': 0,
 'nutty': 0,
 'umami': 0,
 'neutral': 0,
 'rancio': 0,
 'roast': 0,
 'mineral': 0,
 'grain': 0,
 'harsh': 0,
 'earthy': 0,
 'woody': 0,
 'floral': 3,
 'fruity': 3,
 'full_bodied': 3,
 'herbal': 1,
 'oily': 2,
 'peaty': 0,
 'rich': 3,
 'salty': 0,
 'smoky': 2,
 'spicy': 4,
 'sweet': 2,
 'tart': 1,
 'vanilla': 2,
 'id': '5',
 'rating': 95,
 'category': 'Whiskey',
 'tasting_notes': 'This rye beast is crammed with so many spices that a copper label is fitted to the bottle to brace it from bulging. Fruity spring flowers and freshly baked rye bread flood from the whisky. And not just any rye bread, but the kind they teach in a prison baking class. The robust rye spices and bottomless oak rumble on the palate with a delicate caramel that pulls its own weight. The Energizer Bunny’s mentor is the endless finish. An instant classic.'}

### Multi-Hot Encoding the Labels

In [192]:
df = pd.DataFrame(new_data)
df_text = df.iloc[:,-1:]
df_label = df.iloc[:, 0:-4] 

In [193]:
df = pd.DataFrame(df['id']).join(df_text)
df = df.join(df_label)
df.head()

,id,tasting_notes,briny,chemical,bitter,juniper,hogo,nutty,umami,neutral,rancio,roast,mineral,grain,harsh,earthy,woody,floral,fruity,full_bodied,herbal,oily,peaty,rich,salty,smoky,spicy,sweet,tart,vanilla
0,5,This rye beast is crammed with so many spices ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,1,2,0,3,0,2,4,2,1,2
1,6,The aromatics of this brandy have a strong app...,0,0,0,0,0,2,0,0,1,4,0,0,0,1,5,1,4,0,1,3,0,4,0,1,0,5,0,0
2,7,Sweet spices and floral aromas are found as yo...,0,0,1,3,0,0,0,0,0,0,0,0,0,3,0,3,2,0,1,3,0,4,0,0,3,2,0,0
3,11,"The nose is floral, with honey, hay, citrus ze...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,3,3,2,0,3,0,1,4,3,2,3
4,12,The nose is at first difficult to find behind ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,0,2,0,1,2,2,1,2


In [194]:
df_labels = pd.get_dummies(df.iloc[:,2:], columns= df.iloc[:,2:].columns)
col_name = df_labels.columns.tolist()

missed_col = ['chemical_4']
insert_pos = ['chemical_5']

for index, item in enumerate(missed_col):
    col_name.insert(col_name.index(insert_pos[index]), item)  

df_labels = df_labels.reindex(columns=col_name)

df_labels.columns

Index(['briny_0', 'briny_1', 'briny_2', 'briny_3', 'briny_4', 'briny_5',
       'chemical_0', 'chemical_1', 'chemical_2', 'chemical_3',
       ...
       'tart_2', 'tart_3', 'tart_4', 'tart_5', 'vanilla_0', 'vanilla_1',
       'vanilla_2', 'vanilla_3', 'vanilla_4', 'vanilla_5'],
      dtype='object', length=168)

In [195]:
df_info = df.iloc[:,0:2]
df_labels = df_labels.fillna(0, axis=1).astype('uint8')

df2 = pd.concat([df_info, df_labels], axis=1)

print(len(df2.columns.tolist()[2:]))
df2.head(5)

168


,id,tasting_notes,briny_0,briny_1,briny_2,briny_3,briny_4,briny_5,chemical_0,chemical_1,chemical_2,chemical_3,chemical_4,chemical_5,bitter_0,bitter_1,bitter_2,bitter_3,bitter_4,bitter_5,juniper_0,juniper_1,juniper_2,juniper_3,juniper_4,juniper_5,hogo_0,hogo_1,hogo_2,hogo_3,hogo_4,hogo_5,nutty_0,nutty_1,nutty_2,nutty_3,nutty_4,nutty_5,umami_0,umami_1,umami_2,umami_3,umami_4,umami_5,neutral_0,neutral_1,neutral_2,neutral_3,neutral_4,neutral_5,rancio_0,rancio_1,rancio_2,rancio_3,rancio_4,rancio_5,roast_0,roast_1,roast_2,roast_3,roast_4,roast_5,mineral_0,mineral_1,mineral_2,mineral_3,mineral_4,mineral_5,grain_0,grain_1,grain_2,grain_3,grain_4,grain_5,harsh_0,harsh_1,harsh_2,harsh_3,harsh_4,harsh_5,earthy_0,earthy_1,earthy_2,earthy_3,earthy_4,earthy_5,woody_0,woody_1,woody_2,woody_3,woody_4,woody_5,floral_0,floral_1,floral_2,floral_3,floral_4,floral_5,fruity_0,fruity_1,fruity_2,fruity_3,fruity_4,fruity_5,full_bodied_0,full_bodied_1,full_bodied_2,full_bodied_3,full_bodied_4,full_bodied_5,herbal_0,herbal_1,herbal_2,herbal_3,herbal_4,herbal_5,oily_0,oily_1,oily_2,oily_3,oily_4,oily_5,peaty_0,peaty_1,peaty_2,peaty_3,peaty_4,peaty_5,rich_0,rich_1,rich_2,rich_3,rich_4,rich_5,salty_0,salty_1,salty_2,salty_3,salty_4,salty_5,smoky_0,smoky_1,smoky_2,smoky_3,smoky_4,smoky_5,spicy_0,spicy_1,spicy_2,spicy_3,spicy_4,spicy_5,sweet_0,sweet_1,sweet_2,sweet_3,sweet_4,sweet_5,tart_0,tart_1,tart_2,tart_3,tart_4,tart_5,vanilla_0,vanilla_1,vanilla_2,vanilla_3,vanilla_4,vanilla_5
0,5,This rye beast is crammed with so many spices ...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,6,The aromatics of this brandy have a strong app...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0
2,7,Sweet spices and floral aromas are found as yo...,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,11,"The nose is floral, with honey, hay, citrus ze...",1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,12,The nose is at first difficult to find behind ...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


### Exporting as CSV

In [196]:
df2.to_csv('train_data.csv', index=False)

### Basic Exploratory Data Analysis

In [197]:
round(df2.iloc[:,:].describe())

,briny_0,briny_1,briny_2,briny_3,briny_4,briny_5,chemical_0,chemical_1,chemical_2,chemical_3,chemical_4,chemical_5,bitter_0,bitter_1,bitter_2,bitter_3,bitter_4,bitter_5,juniper_0,juniper_1,juniper_2,juniper_3,juniper_4,juniper_5,hogo_0,hogo_1,hogo_2,hogo_3,hogo_4,hogo_5,nutty_0,nutty_1,nutty_2,nutty_3,nutty_4,nutty_5,umami_0,umami_1,umami_2,umami_3,umami_4,umami_5,neutral_0,neutral_1,neutral_2,neutral_3,neutral_4,neutral_5,rancio_0,rancio_1,rancio_2,rancio_3,rancio_4,rancio_5,roast_0,roast_1,roast_2,roast_3,roast_4,roast_5,mineral_0,mineral_1,mineral_2,mineral_3,mineral_4,mineral_5,grain_0,grain_1,grain_2,grain_3,grain_4,grain_5,harsh_0,harsh_1,harsh_2,harsh_3,harsh_4,harsh_5,earthy_0,earthy_1,earthy_2,earthy_3,earthy_4,earthy_5,woody_0,woody_1,woody_2,woody_3,woody_4,woody_5,floral_0,floral_1,floral_2,floral_3,floral_4,floral_5,fruity_0,fruity_1,fruity_2,fruity_3,fruity_4,fruity_5,full_bodied_0,full_bodied_1,full_bodied_2,full_bodied_3,full_bodied_4,full_bodied_5,herbal_0,herbal_1,herbal_2,herbal_3,herbal_4,herbal_5,oily_0,oily_1,oily_2,oily_3,oily_4,oily_5,peaty_0,peaty_1,peaty_2,peaty_3,peaty_4,peaty_5,rich_0,rich_1,rich_2,rich_3,rich_4,rich_5,salty_0,salty_1,salty_2,salty_3,salty_4,salty_5,smoky_0,smoky_1,smoky_2,smoky_3,smoky_4,smoky_5,spicy_0,spicy_1,spicy_2,spicy_3,spicy_4,spicy_5,sweet_0,sweet_1,sweet_2,sweet_3,sweet_4,sweet_5,tart_0,tart_1,tart_2,tart_3,tart_4,tart_5,vanilla_0,vanilla_1,vanilla_2,vanilla_3,vanilla_4,vanilla_5
count,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0,3513.0
mean,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.

In [198]:
tasting_notes_len = []

for item in df2['tasting_notes']:
    tasting_notes_len.append(len(item))
    
print(sum(tasting_notes_len))
    
pd.Series(tasting_notes_len).describe()

1336298


count    3513.000000
mean      380.386564
std       104.685472
min       110.000000
25%       310.000000
50%       366.000000
75%       435.000000
max       903.000000
dtype: float64

In [22]:
pandas_profiling.ProfileReport(df2)

/Users/Henry/opt/anaconda3/envs/simpletransformers/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,32
Number of observations,3451
Total Missing (%),0.0%
Total size in memory,862.9 KiB
Average record size in memory,256.0 B
Numeric,30
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0
